In [1]:
import os
import torch
import numpy as np
import csv

class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []
        self.pretrain_vec = [] # should match index order of words in dict.

    def add_word(self, word, vec=None):
        if vec is None:
            if word not in self.word2idx:
                self.idx2word.append(word)
                self.word2idx[word] = len(self.idx2word) - 1
        else:
            if word not in self.word2idx:
                self.pretrain_vec.append(vec)
                self.idx2word.append(word)
                self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class Corpus(object):
    def __init__(self, path, language):
        self.dictionary = Dictionary()
        if language is not None:
            self.pretrained = self.add_pretrained(os.path.join('', 'wiki.' + language + '.vec'))
        #self.trainid, self.trainlab, self.trainidx = self.tokenize_by_user(os.path.join(path, 'train.csv'),True)
        #self.validid, self.validlab, self.valididx = self.tokenize_by_user(os.path.join(path, 'valid.csv'),False)
        #self.testid, self.testlab, self.testidx = self.tokenize_by_user(os.path.join(path, 'test.csv'),False)
        self.X_train, self.y_train = self.tokenize(os.path.join('', 'train.csv'),True)
        #self.X_valid, self.y_valid = self.tokenize(os.path.join(path, 'valid.csv'),False)
        self.X_test, self.y_test = self.tokenize(os.path.join('', 'test.csv'),False)

    def add_pretrained(self, path):
        assert os.path.exists(path)

        # Add words with pretrained vectors to the dictionary
        # might be weird because no eos was added?
        with open(path, 'r', encoding='utf-8') as f:
            tokens = 0
            for line in f:
                words = line.split()
                if len(words) == 2: #first line
                    continue
                word = words[0]
                vec = words[1:]
                if len(vec) != 300:
                    continue #this skips the space embedding
                #vec = np.array(list(map(float, vec)))
                vec = list(map(float,vec))
                tokens += 1
                
                self.dictionary.add_word(word, vec)
    def tokenize(self, path, header):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            tokens = 0
            prev = None
            if header:
                first = True
            else:
                first = False
            tweet_count = 0
            user_idx = -1
            for row in reader:
                if first:
                    first = False
                    continue
                if len(row) is not 6:
                    continue
                
                tweet = row[0]
                label = row[1]
                if not label.isdigit():
                    continue
                extra = row[2:5] #bio, tweet pic, profile pic, user id
                if row[2] != prev: #new user
                    prev = row[2]
                    tweet_count = 0
                    user_idx += 1

                words = tweet.split()
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)

        # Tokenize file content
        with open(path, 'r', encoding='utf-8') as f:
            x = np.zeros(user_idx+1,dtype='object')
            y = np.zeros(user_idx+1,dtype='int')
            #ids = torch.LongTensor(tokens)
            #idxs = torch.LongTensor(user_idx+1)
            #labels = torch.LongTensor(user_idx+1)
            #print(user_idx+1)
            token = 0
            prev = None

            reader = csv.reader(f)
            if header:
                first = True
            else:
                first = False
            user_idx = -1
            for row in reader:
                if first:
                    first = False
                    continue
                if len(row) is not 6:
                    continue
                
                tweet = row[0]
                label = row[1]
                if not label.isdigit():
                    continue
                extra = row[2:5] #bio, tweet pic, profile pic, user id
                if row[2] != prev:
                    tweet_idx = -1
                    user_idx += 1
                    prev = row[2]
                    y[user_idx] = int(label)
                    x[user_idx] = []
                    #print(token, "NEW USER")
                    #idxs[user_idx] = token
                

                words = tweet.split()
                token = 0
                tweet_idx+=1
                if tweet_idx >=20:
                    #print(tweet_idx)
                    continue
                x[user_idx].append([])
                for word in words:
                    x[user_idx][tweet_idx].append(self.dictionary.word2idx[word])
                    token+=1
                

        return x, y


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

#attention functions

def batch_matmul_bias(seq, weight, bias, nonlinearity=''):
    s = None
    bias_dim = bias.size()
    for i in range(seq.size(0)):
        _s = torch.mm(seq[i], weight) 
        _s_bias = _s + bias.expand(bias_dim[0], _s.size()[0]).transpose(0,1)
        if(nonlinearity=='tanh'):
            _s_bias = torch.tanh(_s_bias)
        _s_bias = _s_bias.unsqueeze(0)
        if(s is None):
            s = _s_bias
        else:
            s = torch.cat((s,_s_bias),0)
    return s.squeeze()

def batch_matmul(seq, weight, nonlinearity=''):
    s = None
    for i in range(seq.size(0)):
        _s = torch.mm(seq[i], weight)
        if(nonlinearity=='tanh'):
            _s = torch.tanh(_s)
        _s = _s.unsqueeze(0)
        if(s is None):
            s = _s
        else:
            s = torch.cat((s,_s),0)
    return s.squeeze()

def attention_mul(rnn_outputs, att_weights):
    attn_vectors = None
    for i in range(rnn_outputs.size(0)):
        h_i = rnn_outputs[i]
        a_i = att_weights[i].unsqueeze(1).expand_as(h_i)
        h_i = a_i * h_i
        h_i = h_i.unsqueeze(0)
        if(attn_vectors is None):
            attn_vectors = h_i
        else:
            attn_vectors = torch.cat((attn_vectors,h_i),0)
    return torch.sum(attn_vectors, 0).unsqueeze(0)

'''
def batch_matmul_bias(seq, weight, bias, nonlinearity=''):
    s = None
    bias_dim = bias.size()
    for i in range(seq.size(0)):
        _s = torch.mm(seq[i], weight) 
        _s_bias = _s + bias.expand(bias_dim[0], _s.size()[0]).transpose(0,1)
        if(nonlinearity=='tanh'):
            _s_bias = torch.tanh(_s_bias)
        _s_bias = _s_bias.unsqueeze(0)
        if(s is None):
            s = _s_bias
        else:
            s = torch.cat((s,_s_bias),0)
    return s.squeeze()
    
def batch_matmul(seq, weight, nonlinearity=''):
    s = None
    for i in range(seq.size(0)):
        _s = torch.mm(seq[i], weight)
        if(nonlinearity=='tanh'):
            _s = torch.tanh(_s)
        _s = _s.unsqueeze(0)
        if(s is None):
            s = _s
        else:
            s = torch.cat((s,_s),0)
    return s.squeeze()

def attention_mul(self, rnn_outputs, att_weights):
    attn_vectors = None
    for i in range(rnn_outputs.size(0)):
        h_i = rnn_outputs[i]
        a_i = att_weights[i].unsqueeze(1).expand_as(h_i)
        h_i = a_i * h_i
        h_i = h_i.unsqueeze(0)
        if(attn_vectors is None):
            attn_vectors = h_i
        else:
            attn_vectors = torch.cat((attn_vectors,h_i),0)
    return torch.sum(attn_vectors, 0).unsqueeze(0)
'''
class AttentionWordRNN(nn.Module):
    
    
    def __init__(self, embeds, batch_size, num_tokens, embed_size, word_gru_hidden, dropout, bidirectional= True):        
        
        super(AttentionWordRNN, self).__init__()
        
        self.batch_size = batch_size
        self.num_tokens = num_tokens
        self.embed_size = embed_size
        self.word_gru_hidden = word_gru_hidden
        self.bidirectional = bidirectional
        self.drop = nn.Dropout(dropout)

        
        
        self.lookup = nn.Embedding(num_tokens, embed_size)

        #init lookup table
        

        
        initrange = 0.1

        k = len(embeds) # the first k indices are pretrained. the rest are unknown
        
        if k is not 0:
            first = np.array(embeds)
            second = np.random.uniform(-initrange,initrange,size=(num_tokens-k,embed_size))
            self.lookup.weight.data.copy_(torch.from_numpy(np.concatenate((first,second),axis=0)))
        else:
            self.lookup.weight.data.uniform_(-initrange, initrange)
        

        if bidirectional == True:
            self.word_gru = nn.GRU(embed_size, word_gru_hidden, bidirectional= True)
            self.weight_W_word = nn.Parameter(torch.Tensor(2* word_gru_hidden,2*word_gru_hidden))
            self.bias_word = nn.Parameter(torch.Tensor(2* word_gru_hidden,1))
            self.weight_proj_word = nn.Parameter(torch.Tensor(2*word_gru_hidden, 1))
        else:
            self.word_gru = nn.GRU(embed_size, word_gru_hidden, bidirectional= False)
            self.weight_W_word = nn.Parameter(torch.Tensor(word_gru_hidden, word_gru_hidden))
            self.bias_word = nn.Parameter(torch.Tensor(word_gru_hidden,1))
            self.weight_proj_word = nn.Parameter(torch.Tensor(word_gru_hidden, 1))
            
        self.softmax_word = nn.Softmax()
        #self.word_gru.data.uniform_(-initrange,initrange)
        self.weight_W_word.data.uniform_(-initrange, initrange)
        self.weight_proj_word.data.uniform_(-initrange,initrange)
        self.bias_word.data.uniform_(-initrange,initrange)

        
        
    def forward(self, embed, state_word):
        # embeddings
        #print(embed)
        embedded = self.drop(self.lookup(embed))
        # word level gru
        #state_word = self.drop(state_word) #idk
        output_word, state_word = self.word_gru(embedded, state_word)
        state_word = self.drop(state_word) #idk
        output_word = self.drop(output_word)
#         print output_word.size()
        word_squish = self.drop(batch_matmul_bias(output_word, self.weight_W_word,self.bias_word, nonlinearity='tanh'))
        word_attn = self.drop(batch_matmul(word_squish, self.weight_proj_word))
        word_attn_norm = self.drop(self.softmax_word(word_attn.transpose(1,0)))
        word_attn_vectors = self.drop(attention_mul(output_word, word_attn_norm.transpose(1,0)))
        return word_attn_vectors, state_word, word_attn_norm
    
    def init_hidden(self):
        if self.bidirectional == True:
            return Variable(torch.zeros(2, self.batch_size, self.word_gru_hidden))
        else:
            return Variable(torch.zeros(1, self.batch_size, self.word_gru_hidden))

class AttentionSentRNN(nn.Module):
    
    
    def __init__(self, batch_size, sent_gru_hidden, word_gru_hidden, n_classes, dropout, bidirectional= True):        
        
        super(AttentionSentRNN, self).__init__()
        
        self.batch_size = batch_size
        self.sent_gru_hidden = sent_gru_hidden
        self.n_classes = n_classes
        self.word_gru_hidden = word_gru_hidden
        self.bidirectional = bidirectional
        
        self.drop = nn.Dropout(dropout)

        initrange = 0.1
        
        
        if bidirectional == True:
            self.sent_gru = nn.GRU(2 * word_gru_hidden, sent_gru_hidden, bidirectional= True)        
            self.weight_W_sent = nn.Parameter(torch.Tensor(2* sent_gru_hidden ,2* sent_gru_hidden))
            self.bias_sent = nn.Parameter(torch.Tensor(2* sent_gru_hidden,1))
            self.weight_proj_sent = nn.Parameter(torch.Tensor(2* sent_gru_hidden, 1))
            self.final_linear = nn.Linear(2* sent_gru_hidden, n_classes)
        else:
            self.sent_gru = nn.GRU(word_gru_hidden, sent_gru_hidden, bidirectional= True)        
            self.weight_W_sent = nn.Parameter(torch.Tensor(sent_gru_hidden ,sent_gru_hidden))
            self.bias_sent = nn.Parameter(torch.Tensor(sent_gru_hidden,1))
            self.weight_proj_sent = nn.Parameter(torch.Tensor(sent_gru_hidden, 1))
            self.final_linear = nn.Linear(sent_gru_hidden, n_classes)
        self.softmax_sent = nn.Softmax()
        self.final_softmax = nn.Softmax()
        self.bias_sent.data.uniform_(-initrange, initrange)
        #self.sent_gru.data.uniform_(-initrange,initrange)
        self.weight_W_sent.data.uniform_(-initrange, initrange)
        self.weight_proj_sent.data.uniform_(-initrange,initrange)
        
        
    def forward(self, word_attention_vectors, state_sent):
        #MANUALLY DROPOUT THE GRU
        #state_word = self.drop(state_sent)
        output_sent, state_sent = self.sent_gru(word_attention_vectors, state_sent)   
        state_word = self.drop(state_sent)
        output_sent = self.drop(output_sent)
        sent_squish = self.drop(batch_matmul_bias(output_sent, self.weight_W_sent,self.bias_sent, nonlinearity='tanh'))
        sent_attn = self.drop(batch_matmul(sent_squish, self.weight_proj_sent))
        sent_attn_norm = self.drop(self.softmax_sent(sent_attn.transpose(1,0)))
        sent_attn_vectors = self.drop(attention_mul(output_sent, sent_attn_norm.transpose(1,0)))    
        # final classifier
        final_map = self.final_linear(sent_attn_vectors.squeeze(0))
        return F.log_softmax(final_map), state_sent, sent_attn_norm
    
    def init_hidden(self):
        if self.bidirectional == True:
            return Variable(torch.zeros(2, self.batch_size, self.sent_gru_hidden))
        else:
            return Variable(torch.zeros(1, self.batch_size, self.sent_gru_hidden))

In [3]:
#import model
#import data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import sklearn.metrics

dropout=0
my_batch=64
lang='en'
datapath = './data/'+lang
corpus = Corpus(datapath, lang)
ntokens = len(corpus.dictionary)
pretrain = corpus.dictionary.pretrain_vec

word_attn = AttentionWordRNN(embeds=pretrain, batch_size=my_batch, num_tokens=ntokens, embed_size=300, 
                             word_gru_hidden=100, dropout=dropout, bidirectional= True)

sent_attn = AttentionSentRNN(batch_size=my_batch, sent_gru_hidden=100, word_gru_hidden=100, 
                             n_classes=2, dropout=dropout, bidirectional= True)

def train_data(mini_batch, targets, word_attn_model, sent_attn_model, word_optimizer, sent_optimizer, criterion):
    state_word = word_attn_model.init_hidden().cuda()
    state_sent = sent_attn_model.init_hidden().cuda()
    max_sents, batch_size, max_tokens = mini_batch.size()
    word_optimizer.zero_grad()
    sent_optimizer.zero_grad()
    s = None
    for i in range(max_sents):
        #torch.cuda.empty_cache()
        _s, state_word, _ = word_attn_model(mini_batch[i,:,:].transpose(0,1), state_word)
        if(s is None):
            s = _s
        else:
            s = torch.cat((s,_s),0)            
    y_pred, state_sent, _ = sent_attn_model(s, state_sent)
    loss = criterion(y_pred.cuda(), targets)

    state_word = None
    state_sent = None
    max_sents = None
    batch_size = None
    max_tokens = None 
    mini_batch = None
    torch.cuda.empty_cache()
    loss.backward()
    
    word_optimizer.step()
    sent_optimizer.step()
    
    return loss.data.item()



def get_predictions(val_tokens, word_attn_model, sent_attn_model):
    max_sents, batch_size, max_tokens = val_tokens.size()
    state_word = word_attn_model.init_hidden().cuda()
    state_sent = sent_attn_model.init_hidden().cuda()
    s = None
    for i in range(max_sents):
        _s, state_word, _ = word_attn_model(val_tokens[i,:,:].transpose(0,1), state_word)
        if(s is None):
            s = _s
        else:
            s = torch.cat((s,_s),0)            
    y_pred, state_sent, _ = sent_attn_model(s, state_sent)    
    return y_pred



#learning_rate = 0.001
#momentum = 0.9
#word_optimizer = torch.optim.SGD(word_attn.parameters(), lr=learning_rate, momentum= momentum)
#sent_optimizer = torch.optim.SGD(sent_attn.parameters(), lr=learning_rate, momentum= momentum)
word_optimizer = torch.optim.Adam(word_attn.parameters())
sent_optimizer = torch.optim.Adam(sent_attn.parameters())
criterion = nn.NLLLoss()

word_attn.cuda()
sent_attn.cuda()



def pad_batch(mini_batch):
    mini_batch_size = len(mini_batch)
    max_sent_len = int(np.mean([len(x) for x in mini_batch]))
    max_token_len = int(np.mean([len(val) for sublist in mini_batch for val in sublist]))
    main_matrix = np.zeros((mini_batch_size, max_sent_len, max_token_len), dtype= np.int)
    for i in range(main_matrix.shape[0]):
        for j in range(main_matrix.shape[1]):
            for k in range(main_matrix.shape[2]):
                try:
                    main_matrix[i,j,k] = mini_batch[i][j][k]
                except IndexError:
                    pass
    #return Variable(torch.from_numpy(main_matrix).transpose(0,1))
    return Variable(torch.LongTensor(main_matrix).transpose(0,1))



def test_accuracy_mini_batch(tokens, labels, word_attn, sent_attn):
    y_pred = get_predictions(tokens, word_attn, sent_attn)
    #print("PRED",y_pred)
    _, y_pred = torch.max(y_pred, 1)
    correct = np.ndarray.flatten(y_pred.data.cpu().numpy())
    labels = np.ndarray.flatten(labels.data.cpu().numpy())
    #print("CORR",correct)
    #print("LABELS",labels)
    num_correct = sum(correct == labels)
    return float(num_correct) / len(correct)

def test_accuracy_full_batch(tokens, labels, mini_batch_size, word_attn, sent_attn):
    p = []
    p_nonlinear = []
    l = []
    g = gen_minibatch(tokens, labels, mini_batch_size)
    for token, label in g:
        y_pred = get_predictions(token.cuda(), word_attn, sent_attn)
        #print("BEFORE",y_pred)
        p_nonlinear.append(np.ndarray.flatten(y_pred[:,1].data.cpu().numpy()))
        _, y_pred = torch.max(y_pred, 1)
        #print("AFTER",y_pred)
        p.append(np.ndarray.flatten(y_pred.data.cpu().numpy()))
        l.append(np.ndarray.flatten(label.data.cpu().numpy()))
    p = [item for sublist in p for item in sublist]
    l = [item for sublist in l for item in sublist]
    p_nonlinear = [np.exp(item) for sublist in p_nonlinear for item in sublist]
    p = np.array(p)
    l = np.array(l)
    #print("TOKEN LEN",len(tokens))
    #print("NONLINEAR",p_nonlinear)
    #print("PREDICT",p)
    #print("LABEL",l)
    num_correct = sum(p == l)
    return float(num_correct)/ len(p), sklearn.metrics.roc_auc_score(l, p_nonlinear)

def test_data(mini_batch, targets, word_attn_model, sent_attn_model):    
    state_word = word_attn_model.init_hidden().cuda()
    state_sent = sent_attn_model.init_hidden().cuda()
    max_sents, batch_size, max_tokens = mini_batch.size()
    s = None
    for i in range(max_sents):
        _s, state_word, _ = word_attn_model(mini_batch[i,:,:].transpose(0,1), state_word)
        if(s is None):
            s = _s
        else:
            s = torch.cat((s,_s),0)            
    y_pred, state_sent,_ = sent_attn_model(s, state_sent)
    loss = criterion(y_pred.cuda(), targets)     
    return loss.data.item()

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert inputs.shape[0] == targets.shape[0]
    if shuffle:
        indices = np.arange(inputs.shape[0])
        np.random.shuffle(indices)
    #print(inputs.shape[0] - batchsize+1, batchsize, "HOO")
    for start_idx in range(0, inputs.shape[0] - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]



def gen_minibatch(tokens, labels, mini_batch_size, shuffle= True):
    for token, label in iterate_minibatches(tokens, labels, mini_batch_size, shuffle= shuffle):
        token = pad_batch(token)
        yield token.cuda(), Variable(torch.LongTensor(label), requires_grad= False).cuda()

def check_val_loss(val_tokens, val_labels, mini_batch_size, word_attn_model, sent_attn_model):
    val_loss = []
    for token, label in iterate_minibatches(val_tokens, val_labels, mini_batch_size, shuffle= True):
        val_loss.append(test_data(pad_batch(token).cuda(), Variable(torch.LongTensor(label), requires_grad= False).cuda(), 
                                  word_attn_model, sent_attn_model))
    return np.mean(val_loss)

import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def train_early_stopping(mini_batch_size, X_train, y_train, X_test, y_test, word_attn_model, sent_attn_model, 
                         word_attn_optimiser, sent_attn_optimiser, loss_criterion, num_epoch, 
                         print_val_loss_every = 1000, print_loss_every = 50):
    #for i in word_attn_model.parameters():
        #print(i.data, "PARAM")
    max_eval_acc = 0
    max_train_acc = 0
    max_eval_aucroc = 0
    max_train_aucroc = 0
    word_attn_model.train()
    sent_attn_model.train()
    start = time.time()
    loss_full = []
    loss_epoch = []
    accuracy_epoch = []
    loss_smooth = []
    accuracy_full = []
    epoch_counter = 0
    g = gen_minibatch(X_train, y_train, mini_batch_size)
    for i in range(1, num_epoch + 1):
        try:
            word_attn_model.train()
            sent_attn_model.train()
            tokens, labels = next(g)
            loss = train_data(tokens, labels, word_attn_model, sent_attn_model, word_attn_optimiser, sent_attn_optimiser, loss_criterion)
            acc = test_accuracy_mini_batch(tokens, labels, word_attn_model, sent_attn_model)
            accuracy_full.append(acc)
            accuracy_epoch.append(acc)
            loss_full.append(loss)
            loss_epoch.append(loss)
            # print loss every n passes
            if i % print_loss_every == 0:
                print('Loss at %d minibatches, %d epoch,(%s) is %f' %(i, epoch_counter, timeSince(start), np.mean(loss_epoch)))
                print('Accuracy at %d minibatches is %f' % (i, np.mean(accuracy_epoch)))
            # check validation loss every n passes
            if i % print_val_loss_every == 0:
                word_attn_model.eval()
                sent_attn_model.eval()
                val_loss = check_val_loss(X_test, y_test, mini_batch_size, word_attn_model, sent_attn_model)
                print('Average training loss at this epoch..minibatch..%d..is %f' % (i, np.mean(loss_epoch)))
                print('Validation loss after %d passes is %f' %(i, val_loss))
                if val_loss > np.mean(loss_full):
                    print('Validation loss is higher than training loss at %d is %f , stopping training!' % (i, val_loss))
                    print('Average training loss at %d is %f' % (i, np.mean(loss_full)))
        except StopIteration:
            epoch_counter += 1
            print('Reached %d epochs' % epoch_counter)
            print('i %d' % i)
            word_attn_model.eval()
            sent_attn_model.eval()
            acc, aucroc = test_accuracy_full_batch(corpus.X_test, corpus.y_test, my_batch, word_attn, sent_attn)
            if acc>max_eval_acc:
                max_eval_acc = acc
            if aucroc>max_eval_aucroc:
                max_eval_aucroc = aucroc
            print("Test accuracy:",acc)
            print("Max test accruacy:",max_eval_acc)
            print("Test aucroc:",aucroc)
            print("Max test aucroc:",max_eval_aucroc)
            word_attn_model.train()
            sent_attn_model.train()
            acc, aucroc = test_accuracy_full_batch(corpus.X_train, corpus.y_train, my_batch, word_attn, sent_attn)
            if acc>max_train_acc:
                max_train_acc = acc
            if aucroc>max_train_aucroc:
                max_train_aucroc = aucroc
            print("Train accuracy:",acc)
            print("Max train accruacy:",max_train_acc)
            print("Train aucroc:",aucroc)
            print("Max train aucroc:",max_train_aucroc)
            #if epoch_counter == 1:
                #break
            g = gen_minibatch(X_train, y_train, mini_batch_size)
            loss_epoch = []
            accuracy_epoch = []
            if epoch_counter==20:
                break
    return loss_full

In [4]:
loss_full= train_early_stopping(my_batch, corpus.X_train, corpus.y_train, corpus.X_test, corpus.y_test, word_attn, sent_attn, word_optimizer, sent_optimizer, 
                            criterion, 5000, 1, 1)

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:160: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:216: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:220: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Loss at 1 minibatches, 0 epoch,(0m 1s) is 0.647809
Accuracy at 1 minibatches is 0.750000
Average training loss at this epoch..minibatch..1..is 0.647809
Validation loss after 1 passes is 0.606373
Loss at 2 minibatches, 0 epoch,(0m 2s) is 0.645069
Accuracy at 2 minibatches is 0.710938
Average training loss at this epoch..minibatch..2..is 0.645069
Validation loss after 2 passes is 0.614336
Loss at 3 minibatches, 0 epoch,(0m 3s) is 0.628871
Accuracy at 3 minibatches is 0.713542
Average training loss at this epoch..minibatch..3..is 0.628871
Validation loss after 3 passes is 0.623119
Loss at 4 minibatches, 0 epoch,(0m 4s) is 0.639505
Accuracy at 4 minibatches is 0.695312
Average training loss at this epoch..minibatch..4..is 0.639505
Validation loss after 4 passes is 0.618722
Loss at 5 minibatches, 0 epoch,(0m 5s) is 0.622061
Accuracy at 5 minibatches is 0.706250
Average training loss at this epoch..minibatch..5..is 0.622061
Validation loss after 5 passes is 0.598139
Reached 1 epochs
i 6
Test

Average training loss at this epoch..minibatch..40..is 0.132154
Validation loss after 40 passes is 0.201171
Loss at 41 minibatches, 6 epoch,(0m 47s) is 0.123768
Accuracy at 41 minibatches is 0.971875
Average training loss at this epoch..minibatch..41..is 0.123768
Validation loss after 41 passes is 0.236251
Reached 7 epochs
i 42
Test accuracy: 0.921875
Max test accruacy: 0.921875
Test aucroc: 0.9554499554499555
Max test aucroc: 0.9576023391812866
Train accuracy: 0.9875
Max train accruacy: 0.9875
Train aucroc: 0.997942789968652
Max train aucroc: 0.997942789968652
Loss at 43 minibatches, 7 epoch,(0m 49s) is 0.090855
Accuracy at 43 minibatches is 0.968750
Average training loss at this epoch..minibatch..43..is 0.090855
Validation loss after 43 passes is 0.210671
Loss at 44 minibatches, 7 epoch,(0m 50s) is 0.065009
Accuracy at 44 minibatches is 0.984375
Average training loss at this epoch..minibatch..44..is 0.065009
Validation loss after 44 passes is 0.231206
Loss at 45 minibatches, 7 epoch,

Loss at 75 minibatches, 12 epoch,(1m 26s) is 0.000151
Accuracy at 75 minibatches is 1.000000
Average training loss at this epoch..minibatch..75..is 0.000151
Validation loss after 75 passes is 0.359492
Validation loss is higher than training loss at 75 is 0.359492 , stopping training!
Average training loss at 75 is 0.218450
Loss at 76 minibatches, 12 epoch,(1m 27s) is 0.000135
Accuracy at 76 minibatches is 1.000000
Average training loss at this epoch..minibatch..76..is 0.000135
Validation loss after 76 passes is 0.319328
Validation loss is higher than training loss at 76 is 0.319328 , stopping training!
Average training loss at 76 is 0.215038
Loss at 77 minibatches, 12 epoch,(1m 28s) is 0.000128
Accuracy at 77 minibatches is 1.000000
Average training loss at this epoch..minibatch..77..is 0.000128
Validation loss after 77 passes is 0.370132
Validation loss is higher than training loss at 77 is 0.370132 , stopping training!
Average training loss at 77 is 0.211731
Reached 13 epochs
i 78
Te

Train accuracy: 1.0
Max train accruacy: 1.0
Train aucroc: 1.0
Max train aucroc: 1.0
Loss at 103 minibatches, 17 epoch,(1m 58s) is 0.000024
Accuracy at 103 minibatches is 1.000000
Average training loss at this epoch..minibatch..103..is 0.000024
Validation loss after 103 passes is 0.477298
Validation loss is higher than training loss at 103 is 0.477298 , stopping training!
Average training loss at 103 is 0.160115
Loss at 104 minibatches, 17 epoch,(1m 59s) is 0.000038
Accuracy at 104 minibatches is 1.000000
Average training loss at this epoch..minibatch..104..is 0.000038
Validation loss after 104 passes is 0.410697
Validation loss is higher than training loss at 104 is 0.410697 , stopping training!
Average training loss at 104 is 0.158275
Loss at 105 minibatches, 17 epoch,(2m 0s) is 0.000030
Accuracy at 105 minibatches is 1.000000
Average training loss at this epoch..minibatch..105..is 0.000030
Validation loss after 105 passes is 0.449444
Validation loss is higher than training loss at 10

In [5]:
#print(test_accuracy_full_batch(corpus.X_test, corpus.y_test, my_batch, word_attn, sent_attn))

In [6]:
#test_accuracy_full_batch(corpus.X_train, corpus.y_train, my_batch, word_attn, sent_attn)